# Input

In [1]:
target_column = "RecommendHiring"

In [2]:
# Parameters
target_column = "Colleague"


# Constants

In [3]:
import os
import numpy as np
from sklearn.metrics import make_scorer

SCRIPT_DIR = os.path.join(os.getcwd(),)

SAVED_MODELS_PATH="./regression_models/saved_models"

DROPPED_LEXICAL_COLUMNS = [
    "Swear",
    "Numbers",
    "Inhibition",
    "Preceptual",
    "Anxiety",
    "Anger",
    "Sadness",
    "Work",
    "Articles",
    "Verbs",
    "Adverbs",
    "Prepositions",
    "Conjunctions",
    "Negations",
]

facial_features = [
    "average_inner_brow_height",
    "average_outer_brow_height",
    "eye_open",
    "inner_lip_height",
    "lip_corner_distance",
    "outer_lip_height",
    "smile",
    "pitch",
    "roll",
    "yaw",
]
stats = ["max", "median", "min", "std", "mean"]
DROPPED_FACIAL_FEATURES = [
    f"{feature}_{stat}" for feature in facial_features for stat in stats
]


DROPPED_PROSODIC_COLUMNS = []

ALREADY_NORMALIZED_FEATURES = [
    "average_outer_brow_height_mean",
    "average_inner_brow_height_mean",
    "eye_open_mean",
    "inner_lip_height_mean",
    "inner_lip_height_mean",
    "lip_corner_distance_mean",
    "average_outer_brow_height_std",
    "average_inner_brow_height_std",
    "eye_open_std",
    "outer_lip_height_std",
    "inner_lip_height_std",
    "lip_corner_distance_std",
    "average_outer_brow_height_min",
    "average_inner_brow_height_min",
    "eye_open_min",
    "outer_lip_height_min",
    "inner_lip_height_min",
    "lip_corner_distance_min",
    "average_outer_brow_height_max",
    "average_inner_brow_height_max",
    "eye_open_max",
    "outer_lip_height_max",
    "inner_lip_height_max",
    "lip_corner_distance_max",
    "average_outer_brow_height_median",
    "average_inner_brow_height_median",
    "eye_open_median",
    "outer_lip_height_median",
    "inner_lip_height_median",
    "lip_corner_distance_median",
]  # these are already in [0, 1]

MUST_KEEP_FEATURES = [
    "pause_duration_avg",
    "average_outer_brow_height_mean",
    "average_inner_brow_height_mean",
    "outer_lip_height_mean",
    "Duration/Filler Words",
]


GROUPS_COLUMN = "cleaned_ids"
INDEX_COLUMN = "participant_id"


def pearson_corr(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]


SCORING_METRICS = {
    "r2": "r2",
    "mae": "neg_mean_absolute_error",
    "pearson": make_scorer(pearson_corr),  # Pearson Correlation Coefficient
}


MUST_KEEP_FEATURES = [
    # "pause_duration_avg",
    # "average_outer_brow_height_mean",
    # "average_inner_brow_height_mean",
    # "outer_lip_height_mean",
    "Duration/Filler Words",
]

PIPELINE_PARAMS = {'feature_selection__estimator__alpha': 0.057376790661083456, 'svr__C': 0.655379988356498, 'svr__gamma': 0.02784736494309893, 'svr__epsilon': 0.2617249201838037, 'svr__kernel': 'rbf'}
HYPERPARAMETER_TUNING_ENABLED = True

# Data Preprocessing

## Import Datasets

In [4]:
import pandas as pd
import os

features_df = pd.read_csv(os.path.join(SCRIPT_DIR, "datasets", "add.csv"))
features_df = features_df.set_index("participant_id")

labels_df = pd.read_csv(
    os.path.join(os.path.join(SCRIPT_DIR,  "datasets", "turker_scores_full_interview.csv"))
)
labels_df = labels_df.set_index("Participant")
labels_df = labels_df.loc[labels_df["Worker"] == "AGGR"]

features_df.index = features_df.index.str.lower()
labels_df.index = labels_df.index.str.lower()
indexed_combined_df = features_df.join(labels_df[[target_column]], how="left")
combined_df = indexed_combined_df.reset_index(drop=True)

# Model

## Split Data

In [5]:
X = combined_df.drop(columns=[target_column, GROUPS_COLUMN])
y = combined_df[target_column]

## Pipeline Creation

In [6]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso, LassoCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GroupKFold, cross_val_score, cross_validate
import sys

sys.path.append("..")
from models.domain_aware_selector import DomainAwareSelector

groups_column = combined_df[GROUPS_COLUMN].astype(str).values


preprocessor = ColumnTransformer(
    [
        ('dropper', 'drop', DROPPED_FACIAL_FEATURES + 
                            DROPPED_LEXICAL_COLUMNS + 
                            DROPPED_PROSODIC_COLUMNS)
    ],
    remainder='passthrough'
)
unfitted_pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ("imputer", SimpleImputer(strategy="mean")),  # NaN imputation
        ("scaler", StandardScaler()),
        ("feature_selection", SelectFromModel(estimator=Lasso(max_iter=50000), )),
        # ('feature_selection', DomainAwareSelector(
        #     must_keep_features=MUST_KEEP_FEATURES,
        #     selector=SelectFromModel(lasso_feature_selection_model, max_features=10),
        # )),
        ("svr", SVR(kernel="rbf")),
    ]
)

## Hyperparameter Tuning

In [7]:
import optuna
from sklearn.base import clone
from sklearn.model_selection import GroupShuffleSplit, cross_val_score
import numpy as np

def objective(trial):
    pipeline_clone = clone(unfitted_pipeline)  # Clone pipeline for thread safety
    
    params = {
        "feature_selection__estimator__alpha": trial.suggest_float(
        "feature_selection__estimator__alpha", 1e-4, 0.1, log=True  # Adjusted lower bound
        ),
        "svr__C": trial.suggest_float("svr__C", 0.01, 100, log=True),
        "svr__gamma": trial.suggest_float("svr__gamma", 1e-3, 1e1, log=True),
        "svr__epsilon": trial.suggest_float("svr__epsilon", 0.01, 0.5),
        "svr__kernel": trial.suggest_categorical("svr__kernel", ["rbf"
                                                                #  , "poly"
                                                                 ]),
    }
    
    # if params["svr__kernel"] == "poly":
    #     params["svr__degree"] = trial.suggest_int("svr__degree", 2, 3)  # Reduced from 5
    #     params["svr__coef0"] = trial.suggest_float("svr__coef0", 0.0, 0.5)  # Narrower range
        
    pipeline_clone.set_params(**params)
    
    mc_cv_tuning = GroupShuffleSplit(n_splits=20, test_size=0.2, random_state=42)
    
    scores = cross_val_score(
        pipeline_clone, X, y, cv=mc_cv_tuning, groups=groups_column, n_jobs=1
    )

    return np.mean(scores)

if HYPERPARAMETER_TUNING_ENABLED:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, n_jobs=-1, timeout=4*60)
    
    print("Best hyperparameters:", study.best_params)
    print(f"Best R² score: {study.best_value:.4f}")

/Users/bassel27/personal_projects/hireverse/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-08 00:45:45,495] A new study created in memory with name: no-name-17051f0c-4483-4676-8323-6f1550ec9d41


[I 2025-04-08 00:45:46,427] Trial 3 finished with value: 0.07845514124275874 and parameters: {'feature_selection__estimator__alpha': 0.0020662523858685565, 'svr__C': 1.8656989848912469, 'svr__gamma': 0.014023063763186953, 'svr__epsilon': 0.07600777242830688, 'svr__kernel': 'rbf'}. Best is trial 3 with value: 0.07845514124275874.


[I 2025-04-08 00:45:46,429] Trial 4 finished with value: 0.10705542655939064 and parameters: {'feature_selection__estimator__alpha': 0.0808499083806953, 'svr__C': 2.7140548097470516, 'svr__gamma': 0.025180540423057517, 'svr__epsilon': 0.3520361852418756, 'svr__kernel': 'rbf'}. Best is trial 4 with value: 0.10705542655939064.


[I 2025-04-08 00:45:46,500] Trial 2 finished with value: -0.14502044698656943 and parameters: {'feature_selection__estimator__alpha': 0.003476005102927311, 'svr__C': 0.05807546909287261, 'svr__gamma': 4.491644364897645, 'svr__epsilon': 0.08128361665408501, 'svr__kernel': 'rbf'}. Best is trial 4 with value: 0.10705542655939064.


[I 2025-04-08 00:45:46,546] Trial 0 finished with value: 0.07716002194844226 and parameters: {'feature_selection__estimator__alpha': 0.000837558133035314, 'svr__C': 99.31980011402402, 'svr__gamma': 0.04581147904428183, 'svr__epsilon': 0.36091783389565285, 'svr__kernel': 'rbf'}. Best is trial 4 with value: 0.10705542655939064.


...

[I 2025-04-08 00:45:59,511] Trial 95 finished with value: 0.20347476302934756 and parameters: {'feature_selection__estimator__alpha': 0.0005815630962529162, 'svr__C': 1.0921814108091665, 'svr__gamma': 0.003965685792645633, 'svr__epsilon': 0.4307127290867892, 'svr__kernel': 'rbf'}. Best is trial 86 with value: 0.21335510004719574.


[I 2025-04-08 00:45:59,690] Trial 99 finished with value: 0.17520814232544688 and parameters: {'feature_selection__estimator__alpha': 0.0006386634694190589, 'svr__C': 1.1988825353540022, 'svr__gamma': 0.0022414481939009444, 'svr__epsilon': 0.42289617454698314, 'svr__kernel': 'rbf'}. Best is trial 86 with value: 0.21335510004719574.


[I 2025-04-08 00:45:59,850] Trial 94 finished with value: 0.19201041131532925 and parameters: {'feature_selection__estimator__alpha': 0.00029285914074015854, 'svr__C': 0.899408003893454, 'svr__gamma': 0.004541595702917719, 'svr__epsilon': 0.4359661688264824, 'svr__kernel': 'rbf'}. Best is trial 86 with value: 0.21335510004719574.


[I 2025-04-08 00:45:59,961] Trial 98 finished with value: 0.16705275648207601 and parameters: {'feature_selection__estimator__alpha': 0.0003485643784588591, 'svr__C': 1.2154509285976414, 'svr__gamma': 0.002137330576625067, 'svr__epsilon': 0.4239952300218971, 'svr__kernel': 'rbf'}. Best is trial 86 with value: 0.21335510004719574.


Best hyperparameters: {'feature_selection__estimator__alpha': 0.0012693428109479302, 'svr__C': 1.08933256186536, 'svr__gamma': 0.007383880324670445, 'svr__epsilon': 0.4747279409436811, 'svr__kernel': 'rbf'}
Best R² score: 0.2134


## Store Trained Model

In [8]:
from sklearn import clone

if HYPERPARAMETER_TUNING_ENABLED:
    unfitted_pipeline.set_params(**study.best_params)
else:
    unfitted_pipeline.set_params(**PIPELINE_PARAMS)
fitted_pipeline = clone(unfitted_pipeline)
fitted_pipeline.fit(X, y)

/Users/bassel27/personal_projects/hireverse/venv/lib/python3.13/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dropper', 'drop',
                                                  ['average_inner_brow_height_max',
                                                   'average_inner_brow_height_median',
                                                   'average_inner_brow_height_min',
                                                   'average_inner_brow_height_std',
                                                   'average_inner_brow_height_mean',
                                                   'average_outer_brow_height_max',
                                                   'average_outer_brow_height_median',
                                                   'average_out...
                                                   'outer_lip_height_max',
                                                   'outer_lip_height_median',
                                                   'outer_lip_height_min',
                                                   'outer_lip_height_std',
                                                   'outer_lip_height_mean', ...])])),
                ('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('feature_selection',
                 SelectFromModel(estimator=Lasso(alpha=0.0012693428109479302,
                                                 max_iter=50000))),
                ('svr',
                 SVR(C=1.08933256186536, epsilon=0.4747279409436811,
                     gamma=0.007383880324670445))])

## Feature Selection Results

In [9]:
preprocessor = fitted_pipeline.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()    # after preprocessing

feature_selector = fitted_pipeline.named_steps['feature_selection']
selected_mask = feature_selector.get_support()

selected_features = feature_names[selected_mask]
unselected_features = feature_names[~selected_mask]

print(f"Number of Selected features ({len(selected_features)}):")
print(f"Selected features: {selected_features}")
print(f"Unselected features: {unselected_features}")

Number of Selected features (39):
Selected features: ['remainder__f0_mean' 'remainder__f0_min' 'remainder__f0_range'
 'remainder__f0_sd' 'remainder__intensity_mean' 'remainder__intensity_min'
 'remainder__intensity_max' 'remainder__intensity_sd' 'remainder__f1_mean'
 'remainder__f1_sd' 'remainder__f2_mean' 'remainder__f2_sd'
 'remainder__f3_mean' 'remainder__f3_sd' 'remainder__f2_f1_mean'
 'remainder__f3_f1_mean' 'remainder__f2_f1_sd' 'remainder__f3_f1_sd'
 'remainder__jitter' 'remainder__shimmer' 'remainder__percent_unvoiced'
 'remainder__percent_breaks' 'remainder__pause_duration_max'
 'remainder__pause_duration_avg' 'remainder__duration'
 'remainder__Total Words' 'remainder__Unique Words'
 'remainder__Filler Words' 'remainder__Audio Duration (s)'
 'remainder__Duration/Total Words' 'remainder__Duration/Unique Words'
 'remainder__Individual' 'remainder__We' 'remainder__They'
 'remainder__PosEmotion' 'remainder__NegEmotion' 'remainder__Cognitive'
 'remainder__Relativity' 'remainder__Qu

# Monte Carlo Cross Validation

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score

scoring = {
    "r2": make_scorer(r2_score),
    "pearson": make_scorer(pearson_corr)
}

results = cross_validate(
    unfitted_pipeline,
    X,
    y,
    cv=GroupShuffleSplit(n_splits=1000, test_size=0.2, random_state=42),
    groups=groups_column,
    scoring=scoring,
    n_jobs=-1,
)

r2_scores = results["test_r2"]
pearson_scores = results["test_pearson"]
# Report the average performance and variability
avg_r2_score = np.mean(r2_scores)
print(
    f"Mean R² Score: {avg_r2_score:.2f} (±{np.std(r2_scores):.2f})"
)
avg_perason_score = np.mean(pearson_scores)
print(
    f"Mean Pearson Correlation: {avg_perason_score:.2f} (±{np.std(pearson_scores):.2f})"
)

# Mean R² Score: 0.20 (±0.23)
# Mean Pearson Correlation: 0.53 (±0.16)

Mean R² Score: 0.14 (±0.22)
Mean Pearson Correlation: 0.47 (±0.17)


# Save the Model

In [11]:
from joblib import dump, load

dump(fitted_pipeline, os.path.join(SAVED_MODELS_PATH,f'{target_column}.joblib'))

['./regression_models/saved_models/Colleague.joblib']